In [17]:
import pandas as pd
import numpy as np

In [18]:
# Load data, ver.10.01
pwt = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

In [19]:
# Filter and select relevant columns, 1001
data = pwt.loc[pwt['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'country', 'rgdpna', 'rkna', 'emp','avh', 'labsh','rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

In [20]:
# Calculate additional columns

data['L'] = data['emp'] # L = emp
#data['L'] = data['emp'] * data['avh']  # L = emp * avh

data['ln_y'] = np.log(data['rgdpna'] / data['L']) # ln(Y/L)
data['ln_k'] = np.log(data['rkna'] / data['L'])  # ln(K/L)
data['alpha'] = 1 - data['labsh']  # alpha
data['alpha_x_ln_k'] = data['alpha'] * data['ln_k']  # alpha * ln(K/L)

In [21]:
# Order by year
data = data.sort_values('year')

In [22]:
# Group by country
grouped_data = data.groupby('country')

In [23]:
# Calculate growth rates, etc.
data['g_rate'] = (grouped_data['ln_y'].diff() * 100)  # Growth rate of GDP 
data['cap_d'] = (grouped_data['alpha_x_ln_k'].diff() * 100)  # Capital Deepening (alpha is included)
#data['cap_d'] = data['alpha'] * (grouped_data['ln_k'].diff() * 100)  # Capital Deepening (alpha is not included)

data['tfp_g'] = data['g_rate'] - data['cap_d']  # TFP Growth

In [24]:
# Remove missing values
data = data.dropna()

In [25]:
# Calculate summary statistics
summary = data.groupby('country').agg({'g_rate': 'mean','tfp_g': 'mean','cap_d': 'mean'})

In [26]:
# Calculate additional summary statistics
summary['Growth Rate'] = summary['g_rate']
summary['TFP Growth'] = summary['tfp_g']
summary['Capital Deepening'] = summary['cap_d']

summary['TFP Share'] = summary['TFP Growth'] /summary['Growth Rate']
summary['Capital Share'] = summary['Capital Deepening'] / summary['Growth Rate']

In [27]:
# Caluculate average
average_growth_rate = summary['Growth Rate'].mean()
average_tfp_growth = summary['TFP Growth'].mean()
average_capital_deepening = summary['Capital Deepening'].mean()
average_tfp_share = average_tfp_growth / average_growth_rate  # Average of TFP Share
average_capital_share = average_capital_deepening / average_growth_rate  # Average of Capital Share
average_row = pd.DataFrame({'Growth Rate': [average_growth_rate], 'TFP Growth': [average_tfp_growth], 'Capital Deepening': [average_capital_deepening], 'TFP Share': [average_tfp_share], 'Capital Share': [average_capital_share]}, index=['Average'])


In [28]:
# Print output
summary = summary.iloc[:, -5:]
summary = pd.concat([summary, average_row])
summary = summary.round(2)
table = summary.to_string(index = True)
lines = table.split('\n')
table_with_border = '\n'.join(lines[:-1]) + '\n' + '-' * len(lines[-1]) + '\n' + lines[-1]

print(table_with_border)

                Growth Rate  TFP Growth  Capital Deepening  TFP Share  Capital Share
Canada                 0.89        0.65               0.24       0.73           0.27
France                 0.83        0.21               0.62       0.25           0.75
Germany                0.66        0.56               0.10       0.84           0.16
Italy                 -0.04       -0.52               0.48      13.99         -12.99
Japan                  0.71        0.73              -0.01       1.02          -0.02
United Kingdom         1.02       -0.27               1.28      -0.26           1.26
United States          1.54        0.96               0.58       0.63           0.37
------------------------------------------------------------------------------------
Average                0.80        0.33               0.47       0.41           0.59
